In [1]:
import pandas as pd

df_OLS3 = pd.read_csv("dm_test_OLS3_results.csv")

df_OLS = pd.read_csv("dm_test_OLS_results.csv")

df_RF = pd.read_csv("dm_test_results_RF.csv")

df_LightGBM = pd.read_csv("dm_test_results_LightGBM.csv")

df_NN = pd.read_csv("dm_test_results_NN.csv")

# Print shapes of all DataFrames
print("df_OLS3 shape:", df_OLS3.shape)
print("df_OLS shape:", df_OLS.shape)
print("df_RF shape:", df_RF.shape)
print("df_LightGBM shape:", df_LightGBM.shape)
print("df_NN shape:", df_NN.shape)

df_OLS3 shape: (295420, 2)
df_OLS shape: (295420, 2)
df_RF shape: (295420, 2)
df_LightGBM shape: (295420, 2)
df_NN shape: (295420, 2)


In [2]:
import pandas as pd

# Load the data frames
# Note for OLS, I will not use OLS3 in the comparison of the dm-test.
df_OLS = pd.read_csv("dm_test_OLS_results.csv")
df_RF = pd.read_csv("dm_test_results_RF.csv")
df_LightGBM = pd.read_csv("dm_test_results_LightGBM.csv")
df_NN = pd.read_csv("dm_test_results_NN.csv")
# Find mismatches between y_actual columns
mismatch_OLS_RF = set(df_OLS['y_actual']).symmetric_difference(set(df_RF['y_actual']))
mismatch_OLS_LightGBM = set(df_OLS['y_actual']).symmetric_difference(set(df_LightGBM['y_actual']))
mismatch_OLS_NN = set(df_OLS['y_actual']).symmetric_difference(set(df_NN['y_actual']))

# Combine all mismatches
all_mismatches = mismatch_OLS_RF.union(mismatch_OLS_LightGBM).union(mismatch_OLS_NN)

# Print mismatched y_actual values
print("Mismatched y_actual values:", all_mismatches)

Mismatched y_actual values: set()


In [3]:
import pandas as pd

# Load the data frames
df_OLS = pd.read_csv("dm_test_OLS_results.csv")
df_RF = pd.read_csv("dm_test_results_RF.csv")
df_LightGBM = pd.read_csv("dm_test_results_LightGBM.csv")
df_NN = pd.read_csv("dm_test_results_NN.csv")

# Concatenate the columns based on y_actual
merged_df = pd.concat(
    [df_OLS.set_index("y_actual"), 
     df_RF.set_index("y_actual"), 
     df_LightGBM.set_index("y_actual"), 
     df_NN.set_index("y_actual")], 
    axis=1
).reset_index()

# Rename columns for clarity
merged_df.columns = [
    "y_actual", 
    "y_pred_OLS", 
    "y_pred_RF", 
    "y_pred_LightGBM", 
    "y_pred_NN"
]

# Print the merged DataFrame
print(merged_df)


        y_actual  y_pred_OLS  y_pred_RF  y_pred_LightGBM  y_pred_NN
0       0.015382    0.002589   0.002591         0.002262  -0.000309
1       0.005373    0.002701   0.001790         0.002262  -0.000309
2       0.011101    0.002768   0.001790         0.002262  -0.000309
3       0.010453    0.003404   0.003980         0.002976  -0.000309
4      -0.001586    0.003242   0.006321         0.002976   0.001013
...          ...         ...        ...              ...        ...
295415 -0.007854   -0.005159  -0.000575        -0.001075   0.000351
295416  0.006928   -0.005620  -0.000147        -0.001141   0.000163
295417  0.001807   -0.005266   0.000061        -0.001169   0.000233
295418 -0.016529   -0.006263  -0.003326        -0.002591  -0.000030
295419 -0.007918   -0.005606  -0.002557        -0.001872   0.000233

[295420 rows x 5 columns]


In [4]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from itertools import combinations

# Diebold-Mariano Test Function 
def dm_test(e1, e2, h=1):
    d = e1**2 - e2**2
    T = len(d)
    d_bar = np.mean(d)
    if T <= 1:
        return np.nan, np.nan
    if h == 1:
        V_d = np.var(d, ddof=1) / T
    else:
        m = h - 1
        gamma0 = np.sum((d - d_bar)**2) / T
        gamma_sum = 0
        for j in range(1, m+1):
            weight = 1 - j/m
            gamma = np.sum((d[j:] - d_bar) * (d[:-j] - d_bar)) / T
            gamma_sum += weight * gamma
        V_d = (gamma0 + 2 * gamma_sum) / T
    dm_stat = d_bar / np.sqrt(V_d)
    p_value = 2 * (1 - norm.cdf(abs(dm_stat)))
    return dm_stat, p_value

# Extract prediction columns and compute errors
errors = {
    'OLS': merged_df['y_actual'] - merged_df['y_pred_OLS'],
    'RF': merged_df['y_actual'] - merged_df['y_pred_RF'],
    'LightGBM': merged_df['y_actual'] - merged_df['y_pred_LightGBM'],
    'NN': merged_df['y_actual'] - merged_df['y_pred_NN']
}

# Define model names
models = ['OLS', 'RF', 'LightGBM', 'NN']

# Create empty 4x4 matrix
dm_matrix = pd.DataFrame(index=models, columns=models, dtype="object")

# Fill matrix with DM test results
for m1, m2 in combinations(models, 2):
    dm_stat, p_value = dm_test(errors[m1].values, errors[m2].values, h=1)
    dm_matrix.loc[m1, m2] = f"{dm_stat:.3f} (p={p_value:.3f})"
    dm_matrix.loc[m2, m1] = f"{-dm_stat:.3f} (p={p_value:.3f})"  # Sign reflects m2 vs m1

# Display results
print("DM Test Results (Entire Sample)")
print(dm_matrix.fillna("-"))


DM Test Results (Entire Sample)
                        OLS                RF          LightGBM  \
OLS                       -  12.044 (p=0.000)  12.121 (p=0.000)   
RF        -12.044 (p=0.000)                 -   3.644 (p=0.000)   
LightGBM  -12.121 (p=0.000)  -3.644 (p=0.000)                 -   
NN        -12.013 (p=0.000)   8.834 (p=0.000)  10.458 (p=0.000)   

                         NN  
OLS        12.013 (p=0.000)  
RF         -8.834 (p=0.000)  
LightGBM  -10.458 (p=0.000)  
NN                        -  


In [5]:
dm_matrix.fillna("-")

,OLS,RF,LightGBM,NN
OLS,-,12.044 (p=0.000),12.121 (p=0.000),12.013 (p=0.000)
RF,-12.044 (p=0.000),-,3.644 (p=0.000),-8.834 (p=0.000)
LightGBM,-12.121 (p=0.000),-3.644 (p=0.000),-,-10.458 (p=0.000)
NN,-12.013 (p=0.000),8.834 (p=0.000),10.458 (p=0.000),-
